## **CNN in Real Time using Transfer Learning with ResNet50 on CIFAR10 Dataset** 



This app recognize movments, detect the object and tracking after it in real time from camera.                                   
We use **Resnet50** model with **Transfer Learning** on **CIFAR10** Dataset that we trained and perfoms **85% accuracy.**       
After we recognize movment we detect the object by using our model and then we start tracking after it.                         
We store all the detections events in format of date and time and and show it on the screen.                                     
The application using **TensorFlow** with **Keras** by **Python**.

### Import Libraries, Loading model and Camera  ###

In [1]:
import os
import io
import cv2
import h5py
import winsound
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from time import strftime
from collections import deque
from timeit import default_timer as timer
from tensorflow.keras.models import load_model

# Define the labels of CIFAR-10 datasest
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Load our model that trained by 25 epochs on CIFAR dataset
model = load_model(r'C:\\Project CNN In Real Time\cifar10_ResNet50_88_Accuracy.h5')

# Catch frame from webcam
camera = cv2.VideoCapture(r'C:\\Project CNN In Real Time\Videos\Pigeons\Pigeon 6.mp4')
# camera = cv2.VideoCapture(1)

# This path is loaction for the saved images 
outPutPath = (r'C:\\Project CNN In Real Time\Saved Images\Detections Events') 

# This path is location for the saved detections events
txtPath = (r'C:\\Project CNN In Real Time\Saved Images\Detections Events\Detection Events.txt')

# This path is location for the sound file
soundPath = (r'C:\\Project CNN In Real Time\Videos\Sounds\Notification 2.wav') 

# Writing The Headline of the text file
with open(txtPath, 'a') as f:
    f.write("###################### Detection Events #########################")
    f.write('\n')
    f.write('\n')
        
# Print trained model summery
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 256)               25690368  
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                        

### Define The Application Constans Variabels ###

In [2]:
# Define object length variables
MIN_OBJECT_AREA = 1000
MAX_OBJECT_AREA = 8000

# Define the mask by number from 0 to 7
mask_number = 6

# Check if fragment still in tracking boundries every X frames
tracking_check = 10000

# Define variables for hight and width shape to prediction model input
hight, width = 224, 224

# Define variables for hight and width shape of the frames
HEIGH, WIDTH = 250, 400   

# Define the font size as precent from the screen size
FONT_SIZE = ((HEIGH/1000))

# Variables for start rows and cols to put text
FIRST_ROW = int(HEIGH/10) +2
FIRST_COL = int(WIDTH/40) +2
ROWS_SPACE = int(HEIGH/10) -5

# Initializing deque object for center points of the detected object
points = deque(maxlen=50)

# Create backgroung of the main frame
foregroundModel = cv2.createBackgroundSubtractorMOG2()

# Sobel filter to detect vertical changes on image
f1 = np.array([[1, 0, -1],
               [2, 0, -2],
               [1, 0, -1]])

# Initializing Conv2D layer for GRAY input
layer = tf.keras.layers.Conv2D(filters=1,
                               kernel_size=(3, 3),
                               strides=1,
                               padding='same',
                               activation='relu',
                               input_shape=(HEIGH, WIDTH, 1),
                               use_bias=False,
                               kernel_initializer=tf.keras.initializers.constant(f1))


# Define tracker dictionary
tracker_dict = { 'csrt': cv2.TrackerCSRT_create,
                 'goturn': cv2.TrackerGOTURN_create,
                 'kcf' : cv2.TrackerKCF_create,
                 'boosting' : cv2.legacy.TrackerMOSSE_create(),
                 'mil': cv2.TrackerMIL_create,
                 'tld': cv2.legacy.TrackerTLD_create(),
                 'medianflow': cv2.legacy.TrackerMedianFlow_create(),
                 'mosse':cv2.legacy.TrackerMOSSE_create()}


# Initialize our tracker after the object
tracker = tracker_dict['csrt']()

### Define number of masks for different kind of tasks ###
Masks using to recognize movements in frames and detect object 

In [3]:
# Function return the right mask according to mask_number
def mask(frame, last_frame, foreground_frame, mask_number):

    # Create 7 different types of masks to the main frame
    if mask_number == 0:
        mask_frame = absdiff_mask(frame_bgr, last_frame)      
    if mask_number == 1: 
        mask_frame = mask1(frame, foreground_mask) # 15 birds
    if mask_number == 2:  
        mask_frame = mask2(frame, foreground_mask) # 25 birds
    if mask_number == 3:
        mask_frame = mask3(frame, foreground_mask) # 10 birds - good tracking
    if mask_number == 4:
        mask_frame = mask4(frame, last_frame) # 16 birds - good tracking
    if mask_number == 5:
        mask_frame = mask5(frame, last_frame) # 7 birds
    if mask_number == 6:
        mask_frame = mask6(frame, foreground_mask) # 46 birds
    if mask_number == 7:
        mask_frame = mask7(frame, foreground_mask) # 4 birds

    return mask_frame

# Function create a mask with and Treshold
def absdiff_mask(frame_bgr, last_frame):
      
    # Slicing from tuple only first two elements
    (high, width) = frame_bgr.shape[:2]

    # Converting captured frame to GRAY by OpenCV function  
    frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
    
    # Create one more frame with Gaussian blur
    frame_gray = cv2.GaussianBlur(frame_gray, (25, 25), 0)  
    
    # Converting captured frame to GRAY by OpenCV function        
    last_frame = cv2.cvtColor(last_frame, cv2.COLOR_BGR2GRAY)

    # Create one more frame with Gaussian blur
    last_frame = cv2.GaussianBlur(last_frame, (25, 25), 0)  
    
    # Calculate the difference between two frames
    abs_diff = cv2.absdiff(last_frame, frame_gray)
    
    # Function exclude values that ara more than treshhold = 15 0 and more than 255
    _, gray_mask = cv2.threshold(abs_diff, 5, 255, cv2.THRESH_BINARY)
    
    return gray_mask

# Function create a mask with Sobel filter to detect vertical changes on image
def mask1(frame_bgr, foreground_mask):

    # Slicing from tuple only first two elements
    (high, width) = frame_bgr.shape[:2]

    # Converting captured frame to GRAY by OpenCV function    
    frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
    
    # Reshaping frame to (batch size, rows, columns, channels)
    input_frame = frame_gray.reshape(1, high, width, 1).astype(np.float32)
    
    # Passing GRAY input to the Conv2D layer and to find contours later
    mask = layer(input_frame)
    
    # Converting output feature map from Tensor to Numpy array
    mask_frame = np.array(mask[0, :, :, 0])

    # Using 'clip' function to exclude values that are less than 0 and more than 255
    mask_frame = np.clip(mask_frame, 0, 255).astype(np.uint8) 
        
    return mask_frame

# Function create a mask with Sobel filter and GaussianBlur
def mask2(frame_bgr, foreground_mask):

    # Slicing from tuple only first two elements
    (high, width) = frame_bgr.shape[:2]

    # Converting captured frame to GRAY by OpenCV function    
    frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
    
    # Create one more frame with Gaussian blur
    frame_gray = cv2.GaussianBlur(frame_gray, (25, 25), 0)  
    
    # Reshaping frame to (batch size, rows, columns, channels)
    input_frame = frame_gray.reshape(1, high, width, 1).astype(np.float32)
    
    # Passing GRAY input to the Conv2D layer and to find contours later
    mask = layer(input_frame)
    
    # Converting output feature map from Tensor to Numpy array
    mask_frame = np.array(mask[0, :, :, 0])

    # Using 'clip' function to exclude values that are less than 0 and more than 255
    mask_frame = np.clip(mask_frame, 0, 255).astype(np.uint8) 
        
    return mask_frame

# Function create a mask with Sobel filter and Treshold
def mask3(frame_bgr, foreground_mask):
    
    # Slicing from tuple only first two elements
    (high, width) = frame_bgr.shape[:2]

    # Converting captured frame to GRAY by OpenCV function    
    gray_frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
    
    # Create one more frame with Gaussian blur
    gray_frame = cv2.GaussianBlur(gray_frame, (25, 25), 0)  
    
    # Reshaping frame to (batch size, rows, columns, channels)
    gray_frame = gray_frame.reshape(1, high, width, 1).astype(np.float32)

    # Passing GRAY input to the Conv2D layer and to find contours later    
    mask = layer(gray_frame)
    
    # Converting output feature map from Tensor to Numpy array
    mask_frame = np.array(mask[0, :, :, 0])

    # Using 'clip' function to exclude values that are less than 0 and more than 255
    mask_frame = np.clip(mask_frame, 0, 255).astype(np.uint8)  
    
    # Function exclude values that ara more than treshhold = 15 0 and more than 255
    _, mask_frame = cv2.threshold(mask_frame, 8, 255, cv2.THRESH_BINARY)

    return mask_frame

# Function create a mask with and Treshold
def mask4(frame_bgr, last_frame):
      
    # Slicing from tuple only first two elements
    (high, width) = frame_bgr.shape[:2]

    # Converting captured frame to GRAY by OpenCV function        
    frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
    
    # Create one more frame with Gaussian blur
    frame_gray = cv2.GaussianBlur(frame_gray, (25, 25), 0)  

    # Converting captured frame to GRAY by OpenCV function        
    last_frame = cv2.cvtColor(last_frame, cv2.COLOR_BGR2GRAY)
    
    # Create one more frame with Gaussian blur
    last_frame = cv2.GaussianBlur(last_frame, (25, 25), 0)    

    # Return mask to detect change between two frames   
    abs_diff = cv2.absdiff(last_frame, frame_gray)
    
    # Function exclude values that ara more than treshhold = 15 0 and more than 255
    _, mask_frame = cv2.threshold(abs_diff, 5, 255, cv2.THRESH_BINARY)    
    
    return mask_frame

# Function create a mask with Sobel filter and Treshold
def mask5(frame_bgr, last_frame):
      
    # Slicing from tuple only first two elements
    (high, width) = frame_bgr.shape[:2]
    
    # Converting captured frame to GRAY by OpenCV function            
    frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
    
    # Create one more frame with Gaussian blur
    frame_gray = cv2.GaussianBlur(frame_gray, (25, 25), 0)  

    # Converting captured frame to GRAY by OpenCV function        
    last_frame = cv2.cvtColor(last_frame, cv2.COLOR_BGR2GRAY)
    
    # Create one more frame with Gaussian blur
    last_frame = cv2.GaussianBlur(last_frame, (25, 25), 0)    
    
    # Return mask to detect change between two frames
    abs_diff = cv2.absdiff(last_frame, frame_gray)

    # Reshaping frame to (batch size, rows, columns, channels)    
    gray_frame = abs_diff.reshape(1, high, width, 1).astype(np.float32)
    
    # Passing GRAY input to the Conv2D layer and to find contours later       
    mask = layer(gray_frame)
    
    # Converting output feature map from Tensor to Numpy array
    mask_frame = np.array(mask[0, :, :, 0])

    # Using 'clip' function to exclude values that are less than 0 and more than 255    
    mask_frame = np.clip(mask_frame, 0, 255).astype(np.uint8)  
    
    # Function exclude values that ara more than treshhold = 15 0 and more than 255
    _, mask_frame = cv2.threshold(mask_frame, 70, 255, cv2.THRESH_BINARY)
    
    return mask_frame

# Function create a mask with connectedComponents
def mask6(frame_bgr, foreground_mask):
    
    # Reduce noises
    structuring_element = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    foreground_mask = cv2.morphologyEx(np.float32(foreground_mask), cv2.MORPH_OPEN, structuring_element)

    # Find out connected components and keep only the large components
    num_labels, image_labels = cv2.connectedComponents(np.array(0<foreground_mask, np.uint8))
    
    # Return components larger than threshold
    foreground_mask = keepLargeComponents(image_labels, treshold=1000) 
    
    # Using 'clip' function to exclude values that are less than 0 and more than 255
    foreground_mask = np.clip(foreground_mask, 0, 255).astype(np.uint8) 
    
    # Converting output feature map from Tensor to Numpy array
    foreground_mask = foreground_mask[:, :, np.newaxis]  
    
    return foreground_mask

# Function create a mask with Sobel filter and connectedComponents
def mask7(frame_bgr, foreground_mask):
    
    # Reduce noises
    structuring_element = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    foreground_mask = cv2.morphologyEx(np.float32(foreground_mask), cv2.MORPH_OPEN, structuring_element)

    # Find out connected components and keep only the large components
    num_labels, image_labels = cv2.connectedComponents(np.array(0<foreground_mask, np.uint8))
    
    # Return components larger than threshold
    foreground_mask = keepLargeComponents(image_labels, treshold=0) 
    
    # Using 'clip' function to exclude values that are less than 0 and more than 255
    foreground_mask = np.clip(foreground_mask, 0, 255).astype(np.uint8) 
    
    # Function exclude values that ara more than treshhold = 15 0 and more than 255
    _, foreground_mask = cv2.threshold(foreground_mask, 0, 255, cv2.THRESH_BINARY)     
    
    # Converting output feature map from Tensor to Numpy array
    foreground_mask = foreground_mask[:, :, np.newaxis]  
        
    return foreground_mask

### Define The Application Functions ###

In [4]:
# Function calculate the area around the object
def cont_area(contour, index):
    
    # Get an approximate rectangle coordinates
    (x_min, y_min, box_width, box_height) = cv2.boundingRect(contours[index])
    
    # Calculate the area of the rectangle
    area = box_width*box_height
        
    return area

# Function return empty frame for initlize the main window
def empty_frame(frame_bgr):
    
    empy_frame = np.zeros(frame_bgr.shape, np.uint8)
    empy_frame[:, :, 0] = 255
    empy_frame[:, :, 1] = 255
    empy_frame[:, :, 2] = 255
    
    return empy_frame

# Function return copy of frame
def duplicate_frame(frame):
    
    duplicate = np.zeros(frame.shape, np.uint8)
    duplicate[:, :, 0] = frame[:, :, 0]
    duplicate[:, :, 1] = frame[:, :, 1]
    duplicate[:, :, 2] = frame[:, :, 2]
    
    return duplicate

# Function return 3-Dimension frame
def expands_dimensions(frame):
    
    new_image = np.zeros((HEIGH, WIDTH, 3), np.uint8)
    new_image[:, :, 0] = frame[:, :, 0]
    new_image[:, :, 1] = frame[:, :, 0]
    new_image[:, :, 2] = frame[:, :, 0]    
    
    return new_image
            
# Function checks if the coordinets into the frame boundries
def check_rect_coord(x_min, y_min, box_width, box_height):
    
    # Check if coordinets in the frame boundries
    if 0 < x_min and x_min + box_width < WIDTH and 0 < y_min and y_min + box_height < HEIGH:
        if 0 < box_width and box_width < WIDTH and 0 < box_height and box_height < HEIGH:

            # Rectangle is in the frame 
            return True
        
    # Rectangle is not in the frame        
    return False

# Function return how many mitnutes passed between two detections
def ditections_timer(detection_time, last_time):
                    
    element = last_time
    minute = element[len(element) -2]
    minute = int(minute) * 10
    second = element[len(element) -1]
    second = int(second)
    minutes = minute + second
    last_time_minutes = minutes

    element = detection_time
    minute = element[len(element) -2]
    minute = int(minute) * 10
    second = element[len(element) -1]
    second = int(second)
    minutes = minute + second
    detection_time_minutes = minutes

    time_past = abs(detection_time_minutes - last_time_minutes)
    
    return time_past

# This function remove the components that are smaller than praticular threshold
def keepLargeComponents(image, treshold):
    
    frame = np.zeros(image.shape) < 0 # boolean array
    unique_labels = np.unique(image.flatten()) # find out every unique value that is actually a label 
    
    for label in unique_labels:
        if label == 0: # background
            pass
        else:
            img = (image == label) # save the component
            if treshold < np.sum(img):
                frame = frame | img # save all the components
                
    return np.float32(255*frame)

# Function plot bar chart with scores values
def bar_chart(obtained_scores, classes_names):
    
    # Arranging X axis
    x_positions = np.arange(obtained_scores.size)

    # Creating bar chart
    bars = plt.bar(x_positions, obtained_scores, align='center', alpha=0.6)

    # Highlighting the highest bar
    bars[np.argmax(obtained_scores)].set_color('red')

    # Giving labels to bars along X axis
    plt.xticks(x_positions, classes_names, rotation=25, fontsize=10)

    # Giving names to axes
    plt.xlabel('Class', fontsize=20)
    plt.ylabel('Value', fontsize=20)

    # Giving name to bar chart
    plt.title('Obtained Scores', fontsize=20)

    # Adjusting borders of the plot
    plt.tight_layout(pad=2.5)

    # Initializing object of the buffer
    b = io.BytesIO()

    # Saving bar chart into the buffer
    plt.savefig(b, format='png', dpi=200)

    # Closing plot with bar chart
    plt.close()

    # Moving pointer to the beginning of the buffer
    b.seek(0)

    # Reading bar chart from the buffer
    bar_image = np.frombuffer(b.getvalue(), dtype=np.uint8)

    # Closing buffer
    b.close()

    # Decoding buffer
    bar_image = cv2.imdecode(bar_image, 1)

    # Resize frame to HEIGH X WIDTH
    bar_image = cv2.resize(bar_image, (WIDTH, HEIGH))

    # Returning Numpy array with bar chart
    return bar_image

# Function cut the detected fragment and return it
def cut_fragment(frame_bgr, contours, index):
    
    # Get an approximate rectangle coordinates
    (x_min, y_min, box_width, box_height) = cv2.boundingRect(contours[index])
    
    # Cutting detected fragment from BGR frame
    cut_fragment_bgr_frame = frame_bgr[y_min: y_min + box_height, x_min: x_min + box_width]
    
    # Resize the fragment to the right frame shape in the main window
    cut_fragment_bgr_frame = cv2.resize(cut_fragment_bgr_frame, (WIDTH, HEIGH))

    return cut_fragment_bgr_frame
        
# Function predict model's output from the cutted fragment    
def prediction_model(cut_fragment_bgr):
    
    # Create a copy of the cut_fragment_bgr frame  
    fragment = duplicate_frame(cut_fragment_bgr)

    # Swapping channels from BGR to RGB by OpenCV function
    fragment = cv2.cvtColor(fragment, cv2.COLOR_BGR2RGB)
    
    # Resizing frame to the right shape of the model's input
    fragment = cv2.resize(fragment, (width, hight), interpolation=cv2.INTER_CUBIC)

    # Extending dimension from (height, width, channels) to (1, heigh, width, channels)
    fragment = fragment[np.newaxis, :, :, :]

    # Predict score from model
    scores = model.predict(fragment)

    return scores

# Function drawing rectangle around the predicted object
def drawing_rectangle(frame_bgr, label, contours, index):
    
    # Get an approximate rectangle coordinates
    (x_min, y_min, box_width, box_height) = cv2.boundingRect(contours[index])

    # Drawing bounding box on the current BGR frame
    cv2.rectangle(frame_bgr, (x_min, y_min), (x_min + box_width, y_min + box_height), (0, 255, 0), 3)

    # Putting text with label on the current BGR frame
    cv2.putText(frame_bgr, label, (x_min - 5, y_min - 25), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

    return frame_bgr   

# Function save the frame of the detected event
def save_detection_event(frame_bgr, detection_time, label):

    # Create a copy of time object
    timer = detection_time
    
    # Save detection_time in format that fit to files
    timer = timer[:2] + "-" + timer[3:5] + "-" + timer[6:13] + "-" + timer[14:16] + "-" + timer[17:]

    # Define the name of the image
    image_name = label + ' ' + str(timer)+'.jpg'
    
    # Define the file adress
    finalPath = os.path.join(outPutPath, image_name)
    
    # Save the frame of detection event
    cv2.imwrite(finalPath, frame_bgr)

    # Checking if there is detection
    if 0 < len(label): 
        
        # Order all line to the same length
        if len(label) < len("airplane"):
            label = str(label) + (len("airplane") - len(label))*" "
            
        # Create string of detection event in date-time format
        detection = "Detected " + str(label) + " at: " + detection_time
        
    # Writing the detection event into the file in txtPath location
    with open(txtPath, 'a') as f:
        f.write(detection)
        f.write('\n')
        
# Function create frame that follow the object movement
def drawing_tracking(frame_bgr, contour_box, scores, index, tracking, fps):
    
    # Variable for ain boundries
    shift_left = int(WIDTH/5) +10
    shift_down = int(HEIGH -FIRST_ROW)
    
    # Define the prediction time for fragment
    model_prediction_time = (end - start)
    
    # Get the coordinates of the rectangle around the object
    (x_min, y_min, box_width, box_height) = [int(a) for a in contour_box]
    
    # Getting current center coordinates of the bounding box
    center = (int(x_min + box_width / 2), int(y_min + box_height / 2))

    # Adding current point to the queue
    points.appendleft(center)                        
        
    # Creating image with black background
    track_frame = np.zeros(frame_bgr.shape, np.uint8)

    # Changing background to Black color
    track_frame[:, :, 0] = 0
    track_frame[:, :, 1] = 0
    track_frame[:, :, 2] = 0

    # Visualizing tracker line
    for i in range(1, len(points)):
        
        # If no points collected yet
        if points[i - 1] is None or points[i] is None:
            continue

        # Draw the line between points
        cv2.line(track_frame, points[i - 1], points[i], (50, 200, 50), 2)

    # Adding text with center coordinates of the bounding box
    cv2.putText(track_frame, 'X: {0}'.format(center[0]), (FIRST_COL, FIRST_ROW +5),
                cv2.FONT_HERSHEY_SIMPLEX, FONT_SIZE*4, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.putText(track_frame, 'Y: {0}'.format(center[1]), (FIRST_COL, FIRST_ROW + 35),
                cv2.FONT_HERSHEY_SIMPLEX, FONT_SIZE*4, (255, 255, 255), 2, cv2.LINE_AA)

    # Adding text with time spent for 2D convolution for current frame
    cv2.putText(track_frame, 'Time : ' + '{0:.3f}'.format(model_prediction_time), (FIRST_COL, HEIGH -40),
                cv2.FONT_HERSHEY_SIMPLEX, FONT_SIZE*3, (255, 255, 255), 2, cv2.LINE_AA)  

    # Adding text with score of convolution for current frame
    cv2.putText(track_frame, 'Score : ' + '{0:.3f}'.format(scores[0][index]), (FIRST_COL, HEIGH -15),
                cv2.FONT_HERSHEY_SIMPLEX, FONT_SIZE*3, (255, 255, 255), 2, cv2.LINE_AA)  

    # Adding text with current label on the frame
    cv2.putText(track_frame, "FPS: " + str(fps), (3*shift_left -2, HEIGH -FIRST_ROW), 
                cv2.FONT_HERSHEY_SIMPLEX, FONT_SIZE*4, (255, 255, 255), 3, cv2.LINE_AA)
       
    # If Tracking is on - put text on frame
    if tracking:
        
        # Adding text with tracking status on the frame
        cv2.putText(track_frame, 'Tracking ', (WIDTH -130, FIRST_ROW + 15), 
                    cv2.FONT_HERSHEY_TRIPLEX, FONT_SIZE*3, (50, 200, 50), 1, cv2.LINE_AA)
        
    # Delete the "Tracking" alert from the screen    
    if not tracking:

        track_frame[WIDTH -135:, 0:FIRST_ROW + 15, 0] = 0
        track_frame[WIDTH -135:, 0:FIRST_ROW + 15, 1] = 0
        track_frame[WIDTH -135:, 0:FIRST_ROW + 15, 2] = 0
        
        track_frame[center[1], center[0], 0] = 0
        track_frame[center[1], center[0], 1] = 0
        track_frame[center[1], center[0], 2] = 0
        
        points.clear()

    return track_frame
  
# Function create the information frame in the main window
def info_frame(inf_frame, fps, counter_images_reading, counter_images_processing, counter_images_tracking, time, label):
    
    # Variable for ain boundries
    shift_left = int(WIDTH/5) +10
    shift_down = int(HEIGH -FIRST_ROW)
        
    # Define the font size as 2*FONT_SIZE=(HEIGHT/1000)
    font_size = (FONT_SIZE*2)
    
    # Initilize frame with white background
    inf_frame = empty_frame(inf_frame)
    
    # Variable hold amount images processing in percentage
    processing_percentage = str(int((counter_images_processing * 100) / counter_images_reading))+"%"
    tracking_percentage = str(int((counter_images_tracking * 100) / counter_images_reading))+"%"
    
    # Adding text with right time for current frame
    cv2.putText(inf_frame, strftime("%H:%M:%S"), (WIDTH -shift_left, shift_down), 
                cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 0, 0), 1, cv2.LINE_AA)
    
    # Adding text with right date for current frame
    cv2.putText(inf_frame, strftime("%d/%m/%Y"), (3*shift_left+20, shift_down +20), 
                cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 0, 0), 1, cv2.LINE_AA)  
    
    # Adding text with Model name of the app
    cv2.putText(inf_frame, "Model: ResNet50", (FIRST_COL, FIRST_ROW), 
                cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 0, 0), 1, cv2.LINE_AA)  
    
    # Adding text with the training datset of the app
    cv2.putText(inf_frame, "Dataset: Cifar10", (FIRST_COL, FIRST_ROW +ROWS_SPACE), 
                cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 0, 0), 1, cv2.LINE_AA)  
    
    # Adding text with Camera FPS for current frame
    cv2.putText(inf_frame, 'Camera FPS: ' + '{0:.0f}'.format(fps), (FIRST_COL, FIRST_ROW + (ROWS_SPACE*2)), 
                cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 0, 0), 1, cv2.LINE_AA)  
    
    # Adding text with reading frames counter for current frame
    cv2.putText(inf_frame, 'Reading Frames: ' + '{0:.0f}'.format(counter_images_reading),(FIRST_COL, FIRST_ROW +(ROWS_SPACE*3)),
                cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 0, 0), 1, cv2.LINE_AA) 
    
    # Adding text with time spent for 2D convolution for current frame
    cv2.putText(inf_frame, 'Tracking Images: ' + tracking_percentage, (FIRST_COL, FIRST_ROW +(ROWS_SPACE*4)), 
                cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 0, 0), 1, cv2.LINE_AA) 
    
    # Adding text with processing images counter for current frame
    cv2.putText(inf_frame, 'Processing Images: ' + processing_percentage, (FIRST_COL, FIRST_ROW +(ROWS_SPACE*5)), 
                cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 0, 0), 1, cv2.LINE_AA) 
        
    # Define the next line where the detections text will start
    end_text_line = (FIRST_ROW +(ROWS_SPACE*6)) +5
    
    # Function put text of all detections events on the frame
    plotting_detections(inf_frame, label, time, end_text_line)
    
    return inf_frame

# Function coninue info_frame Function and put text of detection in info frame
def plotting_detections(inf_frame, label, time, start_text_line):

    # Define the font size as 2*FONT_SIZE=(HEIGHT/1000)
    font_size = (FONT_SIZE*2)
    
    # Find max label string length
    max_len = 0
    for i in range(len(detected_labels)):
        if( max_len < len(detected_labels[i])):
            max_len = len(detected_labels[i])
                     
    # Checking if there is detection
    if 0 < len(label): 
        
        # Order all line to the same length
        if len(label) < max_len:
            label = str(label) + (max_len - len(label))*" "
            
        detection = "Detected " + str(label) + " at: " + time

        # Insert first element to log array
        if len(log) == 0:
            log.append(detection)
        else:
            # Check there ara different time of detection event
            if detection != log[-1]:
                  
                # Calculate the how many minutes passed from the last detection    
                time_past_in_minutes = ditections_timer(time, log[-1])
                log.append(detection)
    
    # Line number we start write objects we detected
    line = start_text_line
    
    # Define the end line bounderies
    end_line = (HEIGH - FIRST_ROW -20)
    
    # Scan all the detections object to plot them on the frame
    for i in range(len(log)):
        
        # Variable represent the detection event as a string
        event = log[i]
        
        # Check frames boundaries including text
        if end_line <= line:
            
            # Delete old detection by set pixels to while and initilize line to start
            inf_frame[start_text_line -20:end_line,:] = 255
            
            # Back to line 130
            line = start_text_line
        
        # Adding text with DETECTION EVENT for current frame
        cv2.putText(inf_frame, event, (10, line), cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 0, 255), 1, cv2.LINE_AA) 
        
        # skip to the next line
        line += int(HEIGH/10) - 5  

# Function get 3 frames and collect them to 1 frame 
def collaction_frames(left_frame, mid_frame, right_frame):

    # Insert all frames to array for scan it
    frames = [left_frame, mid_frame, right_frame]

    # Change all frames to 3 chanels    
    for i in range(len(frames)):
        
        # Check if frames[i][3] is exis
        if len(frames[i].shape) < 3:
            
            # Adding 3-Dimension to the image
            frames[i] = frames[i][:,:,np.newaxis]
        
        # Find frames that not 3 channels
        if frames[i].shape[2] != 3:
             
            # Function expand mask's dimension from 1 to 3 odimensions    
            frames[i] = expands_dimensions(frames[i])
                
    # Define frames in the right order
    left_frame = frames[0]
    mid_frame = frames[1]
    right_frame = frames[2]
    
    # Create one window that contain: frame, tracking, mask
    collaction_frame = np.hstack((left_frame, mid_frame, right_frame))
    
    return collaction_frame

# Function Print every detection event 
def print_detections():
    
    # Print every detection event     
    print("\n###################### Detection Events #########################\n")
    for i in range(len(log)):
        print(log[i])
    print("\n###################### Detection Events #########################\n")

# Function print all the app's information on the screen
def print_info(counter_frames_reading, counter_frames_tracking, counter_success_tracking, 
               counter_frames_processing, counter_images_processing, counter_frames_predictions, 
               counter_birds_predictions, counter_fail_predictions, counter_frames_not_processing):
    
    print("\n###################### Application Information ##################\n")
    print("Frame HEIGH: ", HEIGH, "\nFrame WIDTH: ", WIDTH)
    print("Model Input hight: ", hight, "\nModel Input width: ", width)
    print("mask_number: ", mask_number, "\ntracking_check: ", tracking_check)
    print("MIN_OBJECT_AREA: ", MIN_OBJECT_AREA, "\nMAX_OBJECT_AREA: ", MAX_OBJECT_AREA)
    print(" ")
    print("counter_frames_reading ", counter_frames_reading)
    print("counter_frames_tracking ", counter_frames_tracking)
    print("counter_success_tracking ", counter_success_tracking)
    print("counter_frames_processing ", counter_frames_processing)
    print("counter_images_processing ", counter_images_processing)
    print("counter_frames_predictons ", counter_frames_predictions)
    print("counter_birds_predictions ", counter_birds_predictions)
    print("counter_frames_fail_predictions ", counter_fail_predictions)                                
    print("counter_frames_not_processing", counter_frames_not_processing)
    print(" ")
    print("Tracking Frames: ", str(int((counter_success_tracking * 100) / counter_frames_reading))+"%")
    print("Processing Frames: ", str(int((counter_frames_processing * 100) / counter_frames_reading))+"%")
    print("Right Predictions: ", '{0:.0f}'.format((counter_birds_predictions*100)/counter_frames_predictions)+"%")
    print(" ")
    print("###################### Application Information ##################")

# Function writes all the app's information into a text file
def write_info_txt(counter_frames_reading, counter_frames_tracking, counter_success_tracking, 
                   counter_frames_processing, counter_images_processing, counter_frames_predictions, 
                   counter_birds_predictions, counter_fail_predictions, counter_frames_not_processing):
    
    # Stores all the application's information in array and then writes them into a file
    app_info = []
    app_info.append(str("\n###################### Application Information ##################\n"))
    app_info.append(str("counter frames reading: " + str(counter_frames_reading)))
    app_info.append(str("counter frames tracking: " + str(counter_frames_tracking)))
    app_info.append(str("counter success tracking: " + str(counter_success_tracking)))
    app_info.append(str("counter frames processing: " + str(counter_frames_processing)))
    app_info.append(str("counter images processing: " + str(counter_images_processing)))
    app_info.append(str("counter frames predictons: " + str(counter_frames_predictions)))
    app_info.append(str("counter birds predictions: " + str(counter_birds_predictions)))
    app_info.append(str("counter fail predictions: " + str(counter_fail_predictions)))
    app_info.append(str("counter frames not processing: " + str(counter_frames_not_processing)))
    app_info.append(str(" "))
    app_info.append(str("Tracking Frames: "+ str(int((counter_success_tracking * 100) / counter_frames_reading))+"%"))
    app_info.append(str("Processing Frames: "+str(int((counter_frames_processing * 100) / counter_frames_reading))+"%"))
    app_info.append(str("Right Predictions: "+'{0:.0f}'.format((counter_birds_predictions*100)/counter_frames_predictions)+"%"))
    app_info.append(str(" "))
    app_info.append(str("############################## END ################################\n\n"))

    # Writing all application's information into a text file
    with open(txtPath, 'a') as f:
        f.write('\n'.join(app_info))        

### Define Application Variables ###

In [5]:
# Variable hold the prediction label
label = ""

# Variable hold time of last detection
last_time_detection = ""

# Variable for first time detection aim as time right now
detection_time = strftime("%m/%d/%Y %H:%M:%S")

# Initialize FPS variable
FPS = 0

# Variable hold all the detections events
log = []

# Save all the labels we detected
detected_labels = []

# Initialize stop tracking variable
Stop_Tracking = False

# Initialize TRACKING variables
tracking = False

# Initialize FIRST variables
first = True

# Restart timer for FPS
fps_start = timer() 
fps_end = timer()

# Restart prediction's timer
start = timer()
end = timer()

# Initialize counters variables
counter_predictions = 0
counter_frames_saver = 0
counter_fail_tracking = 0
counter_frames_reading = 0
counter_frames_tracking = 0
counter_frames_processing = 0
counter_images_processing = 0
counter_success_tracking = 0
counter_fail_predictions = 0
counter_frames_per_second = 0
counter_birds_predictions = 0
counter_frames_predictions = 0
counter_frames_not_processing = 0

last_boundries = [0, 0, 0, 0] 

# initialize variables
scores = np.array([(0,float(0))])
index = 0

# Capturing first frame from camera
ret, frame_bgr = camera.read()

# Check to camera\video path
if not ret or frame_bgr is None:
    print("Video path not found. ")
else: 
    # Resize the main frame to (HEIGH, WIDTH)
    frame_bgr = cv2.resize(frame_bgr, (WIDTH, HEIGH))

    # Create frame for previos frame_bgr
    last_frame = duplicate_frame(frame_bgr)

    # Define the frame for the first frames in the main window
    scores_frame = empty_frame(frame_bgr)
    tracking_frame = empty_frame(frame_bgr)
    cut_fragment_bgr_frame = empty_frame(frame_bgr)
    demo_frame = empty_frame(frame_bgr)
    mask_frame = np.zeros((HEIGH, WIDTH), np.uint8)

    # Converting captured frame to GRAY by OpenCV function  
    frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)

In [6]:
# Function tracking after objects
def objects_tracking(frame_bgr, tracking, counter_frames_tracking, counter_success_tracking):
    
    # Increase tracking counter
    counter_frames_tracking +=1

    # Get the bounding box from the frame
    (success, contour_box) = tracker.update(frame_bgr)

    # Strart\Keep tracking
    if success:   

        # Save the last contour box to show last tracking location
        last_boundries = contour_box

        # Get the coordinates of the rectangle around the object
        (x, y, w, h) = [int(a) for a in contour_box]

        # Check if coordinates is in the frame boundaries
        if 0 < x and x+w < WIDTH and 0 < y and y+h < HEIGH:   

            # Cut the fragment from the mask frame
            cut_fragment_mask = mask_frame[y:y+h, x:x+w]

            # Checking if tracking is still running after the object or not
            if((counter_frames_reading%5 == 0 and np.sum(cut_fragment_mask) == 0)):      

                # Initializes variable
                Stop_Tracking = True
                
            else:
                
                # Set tracking status ON
                tracking = True

                # Function return a frame with the tracking of the cut fragment
                tracking_frame = drawing_tracking(frame_bgr, contour_box, scores, index, tracking, FPS +1)      

                # Putting text with label on the current BGR frame
                cv2.putText(frame_bgr, label, (x - 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)  
                   
                # Drawing bounding box on the current BGR frame        
                cv2.rectangle(frame_bgr, (x,y), (x+w,y+h), (100,255,0), 2)
                
                # Increase the tracking counter
                counter_success_tracking += 1

                # Variable to keep tracking
                Stop_Tracking = False

    # Return BGR frame with rectangle drawing around the objects
    return frame_bgr, tracking, counter_frames_tracking, counter_success_tracking, Stop_Tracking

### Images Processing and Model Predictions ###

Reading frames from camera frame by frame and recognize movments to detect object in the frame. The detected object is cutted from the original frame and resize to the model input shape. The cutted object sent to model and gives us the label prediction with the score. After we detected the object we drawing green rectangle around it and start to tracking after the object. We display to the user one window that divide to six small windows:

1. The frame that we read from camera with the rectangle around the object and the label above it.
2. Window that contain information of the image processing like model's name, detection event, time, etc'.
3. Window that draw a line that present the tracking of the detected object.
4. Frame of the object we cutted form the original frame and sent to the model.
5. Window that show us graph with all the labels and their score from predicted object.
6. Window that display us the mask we use to detect object and movements.


In [7]:
# Loop reading frame by frame and processing them
while True:
    
    # Capturing frames one-by-one from camera
    ret, frame = camera.read()

    # If the frame was not retrieved then we break the loop
    if not ret or frame is None:
        break

    # Stopping the timer for FPS
    fps_stop = timer()

    # Increasing counter
    counter_frames_reading += 1
    
    # Increasing FPS counter
    counter_frames_per_second += 1
    
    # Define the fps of the loop using cv2 function
    fps = int(camera.get(cv2.CAP_PROP_FPS))
    
    # Resize the main frame to (WIDTH, HEIGH) shape
    frame = cv2.resize(frame, (WIDTH, HEIGH))

    # Create frame for previos frame_bgr  
    frame_bgr = duplicate_frame(frame)

    # Apply the frame to forground model
    foreground_mask = foregroundModel.apply(frame) 
                
    # Function return one from 7 different kinds of masks according to the mask_number              
    mask_frame = mask(frame, last_frame, foreground_mask, mask_number)
       
    # Checking the sum of pixels values to estimate the background
    if(500 < np.sum(mask_frame)):
        
        # Define last frame (using to detect movments)
        last_frame = duplicate_frame(frame_bgr)
            
        # End of if contours       
        if tracking:
                       
            # Increase tracking counter
            counter_frames_tracking +=1
            
            # Get the bounding box from the frame
            (success, contour_box) = tracker.update(frame_bgr)

            # Strart\Keep tracking
            if success:   
            
                # Save the last contour box to show last tracking location
                last_boundries = contour_box
                                
                # Get the coordinates of the rectangle around the object
                (x, y, w, h) = [int(a) for a in contour_box]

                # Check if coordinates is in the frame boundaries
                if 0 < x and x+w < WIDTH and 0 < y and y+h < HEIGH:   
                    
                    # Cut the fragment from the mask frame
                    cut_fragment_mask = mask_frame[y:y+h, x:x+w]
                    
                    # Checking if tracking is still running after the object or not
                    if((counter_frames_reading%5 == 0 and np.sum(cut_fragment_mask) == 0)):      
                        
                        # Initializes variable
                        Stop_Tracking = True
                    else:
                        # Set tracking status ON
                        tracking = True
        
                        # Function return a frame with the tracking of the cut fragment
                        tracking_frame = drawing_tracking(frame_bgr, contour_box, scores, index, tracking, FPS +1)                         

                        # Drawing bounding box on the current BGR frame        
                        cv2.rectangle(frame_bgr, (x,y), (x+w,y+h), (100,255,0), 2)

                        # Putting text with label on the current BGR frame
                        cv2.putText(frame_bgr, label, (x - 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)  

                        # Increase the tracking counter
                        counter_success_tracking += 1
                        
                        # Variable to keep tracking
                        Stop_Tracking = False
                    
            # Stop the tracking after the object    
            if Stop_Tracking:
                
                # Clear all the coordinates from the Deque points
                points.clear()   

                # Initialize Variables to predict the next fragment
                first, tracking = True, False

                # Function return a frame with the tracking of the cut fragment
                tracking_frame = drawing_tracking(frame_bgr, last_boundries, scores, index, tracking, FPS +1)  
                
            # Initializes variable
            Stop_Tracking = True
                
        # End of if tracking
        else:     
            
            # Increase tracking counter
            counter_frames_processing +=1
            
            # Find contsours from the mask to detect objects
            contours, _ = cv2.findContours(mask_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # version 2 of code only draw large change
            if contours and not tracking:

                # Variable for first time recognize a bird
                first = True

                # Variable hold index of contour in contours array
                contour_index = 0

                # Sorted the contours and define the larger first
                contours = sorted(contours, key=cv2.contourArea, reverse=True)

                # Scan all the contours we found in the frame
                for contour in contours:
                    
                    # Return square area of the given contour
                    contour_area = abs(cont_area(contour, contour_index))
                    
                    # Avoid large objects ( max = 10,000)
                    if contour_area < MAX_OBJECT_AREA:
                        
                        # Avoid small objects ( min = 1,000 )
                        if MIN_OBJECT_AREA < contour_area:

                            # Function return cut bgr fragments (will use as input to the model)
                            cut_fragment_bgr_frame = cut_fragment(frame, contours, contour_index)

                            # Predic model output every two frames
                            if counter_frames_reading%2 >= 0:
                                
                                # Increase prediction counter
                                counter_frames_predictions += 1
                                
                                # Current time of detection object
                                detection_time = strftime("%d/%m/%Y %H:%M:%S")  

                                "############################### Start of Model Prediction ###########################"
                                # Measuring classification time
                                start = timer()
                                    
                                # Function return all scores of model predictions
                                scores = prediction_model(cut_fragment_bgr_frame)       

                                # End of Measuring classification time
                                end = timer()
                                "############################### End of Model Prediction #############################"
                        
                                # Finds the lables array index by the max score index of model prediction
                                index = np.argmax(scores)

                                # Define the label for the cut_fragment from labels array
                                label = labels[index]
                                
                                # Function return a bgr frame with rectangle around the cut fragment
                                frame_bgr = drawing_rectangle(frame_bgr, label, contours, contour_index)  
                            
                                # Save detection event in date-time format
                                save_detection_event(frame_bgr, detection_time, label)

                                # Function return a frame with all the labels scores     
                                scores_frame = bar_chart(scores[0], labels)  
                            
                            # Save the label we detected
                            if label not in detected_labels:
                                detected_labels.append(label)          
                                        
                            # Start traking after the detected bird
                            if label == "bird":
                                                               
                                # Clear all the coordinates from the Deque points
                                points.clear()  
                                
                                winsound.PlaySound(soundPath, winsound.SND_FILENAME)

                                # Increasing Birds prediction counter
                                counter_birds_predictions += 1
                                
                                # First time detection (before tracking)
                                if first:
                                    
                                    # Get an approximate rectangle coordinates
                                    (x_min, y_min, box_width, box_height) = cv2.boundingRect(contours[contour_index])
                                        
                                    rectangle = np.array([x_min, y_min, box_width, box_height])
                                    tracker.init(frame_bgr, rectangle)
                                    tracking = True
                                    first = False
                                    break                                   
                                    
                            # If not detected birds        
                            else: 
                                # Increasing fail predictions counter
                                counter_fail_predictions = counter_frames_predictions - counter_birds_predictions
                                                        
                        # End of MAX CONTOUR < contour_area    
                        else:
                            # Skip on contours area small than MIN CONTOUR
                            break                           
                        # End of if\else MIN_OBJECT_AREA < contour_area                         
                    # End of if\else contour_area < MAX_OBJECT_AREA  
                            
                    # Increase the index of contour in the contours array
                    contour_index += 1      
                    
                # End of for contour loop  
                
                # Increasing processing counter
                counter_images_processing += 1
                
            # End of if countrs and not tracking
        # End of if\else tracking
    # if(500 < np.sum(mask_frame))
               
        # Function return frame that contain the app stats and informations
        inf_frame = info_frame(demo_frame, fps, counter_frames_reading, counter_frames_processing, counter_frames_tracking, 
                               detection_time, label)

        # Function return one window that contain (frame_bgr, track_frame, mask_frame)
        upper_window = collaction_frames(frame_bgr, inf_frame, tracking_frame)
        
        # Function return one window that contain (cut_fragment_bgr_frame, scores_frame, mask_frame)
        lower_window = collaction_frames(cut_fragment_bgr_frame, scores_frame, mask_frame)
        
        # Create one window that contain: upper_window and lower_window
        main_window = np.vstack((upper_window, lower_window))
        
        # Plotting all the frames in one window
        cv2.imshow("Main_Window", main_window) 
                
    # End of if(500 < np.sum(mask_frame)):
    else:
        # Increasing the unprocessing counter
        counter_frames_not_processing += 1
                            
        # Function return a frame with the tracking of the cut fragment
        tracking_frame = drawing_tracking(frame_bgr, last_boundries, scores, index, tracking, FPS)  
        
        # Function return frame that contain the app stats and informations
        inf_frame = info_frame(demo_frame, fps, counter_frames_reading, counter_frames_processing, counter_frames_tracking, 
                                detection_time, label)
        
        # Function return one window that contain (frame_bgr, track_frame, mask_frame)
        upper_window = collaction_frames(frame_bgr, inf_frame, tracking_frame)
        
        # Function return one window that contain (cut_fragment_bgr_frame, scores_frame, mask_frame)
        lower_window = collaction_frames(cut_fragment_bgr_frame, scores_frame, mask_frame)
        
        # Create one window that contain: upper_window and lower_window
        main_window = np.vstack((upper_window, lower_window))
    
        # Plotting all the frames in one window
        cv2.imshow("Main_Window", main_window)     
    
    # Print FPS every 1 second
    if 1.0 <= fps_stop - fps_start:
        
        # Define FPS
        FPS = counter_frames_per_second
        
        # Reset FPS counter
        counter_frames_per_second = 0

        # Restart timer for FPS
        fps_start = timer()    
                
    # Function waits for key to be pressed    
    key = cv2.waitKey(1) % 256

    # If 'q' key is pressed then quit from app
    if key == ord('q'):
        break
     
    # if 'i' key is pressed then print all the pixels values
    if key == ord('i'):
        
        # Print the pixel's values sum and last contour area
        print("Sum: ", np.sum(mask_frame))
        print("Area: ", contour_area)
     
    # If 'n' key is pressed then release the tracking and move on
    if key == ord('n'):
        
        # Clear all the coordinates from the Deque points
        points.clear()   

        # Initialize Variables to predict the next fragment
        first, tracking = True, False
                    
        # Function return a frame with the tracking of the cut fragment
        tracking_frame = drawing_tracking(frame_bgr, last_boundries, scores, index, tracking, FPS)  

# Releasing camera
camera.release()

# Destroying all opened OpenCV windows
cv2.destroyAllWindows()

# Print every detection event 
print_detections()

# Print all the app's information 
print_info(counter_frames_reading, counter_frames_tracking, counter_success_tracking, counter_frames_processing, 
           counter_images_processing, counter_frames_predictions, counter_birds_predictions, counter_fail_predictions, 
           counter_frames_not_processing)

# Function write all the app's information into a txt file
write_info_txt(counter_frames_reading, counter_frames_tracking, counter_success_tracking, counter_frames_processing, 
               counter_images_processing, counter_frames_predictions, counter_birds_predictions, counter_fail_predictions, 
               counter_frames_not_processing)


1/1 [==============================] - 0s 88ms/step

###################### Detection Events #########################

Detected bird at: 14/01/2023 15:40:52
Detected bird at: 14/01/2023 15:40:55
Detected bird at: 14/01/2023 15:41:04
Detected bird at: 14/01/2023 15:41:05
Detected bird at: 14/01/2023 15:41:06
Detected cat  at: 14/01/2023 15:41:06
Detected bird at: 14/01/2023 15:41:07
Detected bird at: 14/01/2023 15:41:08
Detected bird     at: 14/01/2023 15:41:14
Detected bird     at: 14/01/2023 15:41:32
Detected bird     at: 14/01/2023 15:41:55

###################### Detection Events #########################


###################### Application Information ##################

Frame HEIGH:  250 
Frame WIDTH:  400
Model Input hight:  224 
Model Input width:  224
mask_number:  6 
tracking_check:  10000
MIN_OBJECT_AREA:  1000 
MAX_OBJECT_AREA:  8000
 
counter_frames_reading  1312
counter_frames_tracking  1287
counter_success_tracking  1274
counter_frames_processing  16
counter_images_proc